## 0. GraphDB 연결하기 
영화 추천 데이터셋 : https://github.com/neo4j-graph-examples/recommendations
* 벡터기반의 검색을 위해서는 텍스트 형태의 데이터가 있어야함 
* 벡터임베딩을 기반으로 검색을 한 후 그 주변에 있는 맥락을 확인하는 그래프형태 
  * 영화 줄거리를 벡터임베딩으로 먼저 찾음
  * 검색된 영화 노드를 기준으로 추가적인 그래프 순회 로직 추가 

In [ ]:
from neo4j import GraphDatabase, basic_auth


driver = GraphDatabase.driver(
  "neo4j://54.209.48.102:7687",
  auth=basic_auth("neo4j", "baby-grain-challenge"))

In [4]:
# 로컬에 있는 .env 를 통해 API key 불러옴 
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from neo4j_graphrag.llm.openai_llm import OpenAILLM

llm = OpenAILLM(model_name="gpt-4o")

## 1. Vector+ Graph 기반 RAG 구현하기

### 1) Embedding, Vector Index 추가하기

In [ ]:
# 영화추천 DB 에는 벡터임베딩이 없으므로, 추가해줌 
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings

embedder = SentenceTransformerEmbeddings(
    model = "all-MiniLM-L6-v2"
)

c:\Users\User\anaconda3\envs\langgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with driver.session() as session:
    result = session.run("MATCH (m:Movie) WHERE m.plot IS NOT NULL RETURN elementId(m) AS id, m.plot AS plot")
    records = result.data()

    for record in records:
        node_id = record["id"] # 노드 고유 ID 추출
        text = record["plot"] # 영화 줄거리 텍스트 추출
        vector = embedder.embed_query(text) # 텍스트를 벡터로 변환 (예: 384 차원)

        session.run("""
        MATCH (m) WHERE elementId(m) = $id
        SET m.embedding = $embedding 
        """, {"id": node_id, "embedding": vector}) # ID로 특정 노드를 찾은 뒤, 해당 노드에 임베딩 속성을 추가, 중괄호는 파라미터 바인딩(SQL Injection 방지지)



In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_vector_index

INDEX_NAME = "plotindex"
DIMENSION = 384

create_vector_index(
    driver, 
    INDEX_NAME, 
    label="Movie", 
    embedding_property="embedding", 
    dimensions=DIMENSION,
    similarity_fn="cosine"
)